**извлечение признаков из текста на естественном языке**

очистка текста и токенизация

_Евгений Борисов <esborisov@sevsu.ru>_

## библиотеки

In [1]:
import re
# import numpy as np
# import numpy.random as rng
import pandas as pd
# from tqdm import tqdm
from tqdm.notebook import tqdm

# np.set_printoptions(precision=2) # вывод на печать чисел до 2 знака
pd.options.display.max_colwidth = 200 

tqdm.pandas()

## тексты

In [2]:
# загружаем тексты
data = pd.read_pickle('../data/news.pkl.gz')
print('записей:',len(data))
data.sample(5)

записей: 3196


,text,tag
3103,СеверИнфо (severinfo.ru) (Вологда) Тундра в ожидании праздника Приближается\nглавный праздник тундры - День Оленя. О подготовке к нему пишет окружная\nгазета ?Няръяна вындер?. Немногие праздники с...,social
434,"Коалиция США нанесла удары по госпиталю в Мосуле\n\n8 декабря 2016 в 9:01\n\nLenta.ru\n\nВозглавляемая США коалиция нанесла удары по госпиталю в Мосуле, который использовали боевики «Исламского го...",politics
677,Компетентное мнение о среднем образовании на английском языке\n\n28 ноября 2016 в 9:00\n\n*На правах рекламы\n\nО чем говорим:\n\n— В каких странах могут учиться белорусские дети\n\n— Дорого и оче...,reclama
1902,"Ковентри (Великобритания), 13 декабря. Британские астрономы открыли экзопланету HAT-P-7 b. В атмосфере небесного тела есть облака из корунда — минерала, разновидностями которого являются рубин и с...",science
2374,"Водитель оставил машину и скрылся с места происшествия.\n\nНа западе Москвы, в районе Раменки, водитель автомобиля Mercedes совершил наезд на двух школьниц. Об этом во вторник, 13 декабря, сообщил...",incident


In [3]:
# длина строк
pd.DataFrame(data['text'].str.len()).describe([.1,.25,.5,.75,.95]).astype(int).T

,count,mean,std,min,10%,25%,50%,75%,95%,max
text,3196,1720,2177,25,370,675,1070,1813,5761,30710


In [4]:
# количество категорий
data['tag'].drop_duplicates().count()

13

In [5]:
# собираем словарь из текстов
def get_vocabulary(ds):
    vcb = [ set(s) for s in ds.tolist() ]
    return sorted(set.union(*vcb))

## очистка и токенизация 

In [6]:
# применяет список замен pat к строке s
def replace_patterns(s,pat):
    if len(pat)<1: return s
    return  replace_patterns( re.sub(pat[0][0],pat[0][1],s), pat[1:] )

# нормализация текста
def string_normalizer(s):
    pat = [
       [r'ё','е'] # замена ё для унификации
       ,[r'</?[a-z]+>',' '] # удаляем xml
       ,[r'[^a-zа-я\- ]+',' '] # оставляем только буквы, пробел и -
       ,[r' -\w+',' '] # удаляем '-й','-тый' и т.п.
       ,[r'\w+- ',' ']
       ,[r' +',' '] # удаляем повторы пробелов
    ]
    return replace_patterns(s.lower(),pat).strip()

data['ctext'] = data['text'].progress_apply(string_normalizer)

  0%|          | 0/3196 [00:00<?, ?it/s]

In [7]:
# разрезаем стоки на слова
def tokenize(s): 
    return [ w for w in s.split(' ') if (len(w)>1) ]

data['ctext'] = data['ctext'].progress_apply( tokenize )

  0%|          | 0/3196 [00:00<?, ?it/s]

In [8]:
vcb =  get_vocabulary( data['ctext'] )
print('словарь %i слов'%(len(vcb)))
# pd.DataFrame( vcb ).to_csv('voc0.txt',index=False,header=False)

словарь 83094 слов


In [9]:
data.sample(3)

,text,tag,ctext
2587,"Полная Луна помешает москвичам увидеть в декабре самый красивый звездопад северного полушария Земли - Геминиды, сообщает пресс-служба Московского планетария.\n\n«С 4 по 17 декабря на ночном небе н...",science,"[полная, луна, помешает, москвичам, увидеть, декабре, самый, красивый, звездопад, северного, полушария, земли, геминиды, сообщает, пресс-служба, московского, планетария, по, декабря, на, ночном, н..."
835,"Новоселы в Гродненском зоопарке: обезьянки-вегетарианцы, сова-сипуха и маленькая альпака\n\n30 ноября 2016 в 9:27\n\nTUT.BY / Фото: Игорь Ремзик\n\nДвух обезьянок необычной окраски зоопарк приобре...",social,"[новоселы, гродненском, зоопарке, обезьянки-вегетарианцы, сова-сипуха, маленькая, альпака, ноября, tut, by, фото, игорь, ремзик, двух, обезьянок, необычной, окраски, зоопарк, приобрел, калининград..."
2281,В связи с неблагоприятными погодными условиями сотрудники ГИБДД приняли решение перекрыть движение по автомагистрали Кемерово – Ленинск-Кузнецкий в обоих направлениях. Временное перекрытие сделано...,incident,"[связи, неблагоприятными, погодными, условиями, сотрудники, гибдд, приняли, решение, перекрыть, движение, по, автомагистрали, кемерово, ленинск-кузнецкий, обоих, направлениях, временное, перекрыти..."


## стеминг

In [10]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords
# from nltk import download as nltk_download
# nltk_download('stopwords')

stopwords = set(nltk_stopwords.words('russian'))
stemmer = SnowballStemmer('russian')

# выкидываем stopwords, выполняем стеминг
def stem(s): 
    return [ stemmer.stem(w) for w in s if w not in stopwords ]

In [11]:
data['ctext'] = data['ctext'].progress_apply( stem )

  0%|          | 0/3196 [00:00<?, ?it/s]

In [12]:
vcb =  get_vocabulary( data['ctext'] )
print('словарь %i слов'%(len(vcb)))
# pd.DataFrame( vcb ).to_csv('voc1.txt',index=False,header=False)

словарь 36381 слов


In [13]:
data.sample(3)

,text,tag,ctext
1831,"Иран и Россия подписали в Тегеране меморандум о сотрудничестве в области нефти и энергетики. Документ, состоящий из 23 пунктов, был подписан в понедельник при участии заместителя министра энергети...",economics,"[ира, росс, подписа, тегеран, меморандум, сотрудничеств, област, нефт, энергетик, документ, состоя, пункт, подписа, понедельник, участ, заместител, министр, энергетик, росс, кирилл, молодцов, замм..."
2874,"Корейский автопроизводитель планировал начать продажи нового Tivoli в России ещё в ноябре, но уже прошла половина декабря, а официальных новостей о новинке по-прежнему нет.\n\nВ российской пресс-с...",auto,"[корейск, автопроизводител, планирова, нача, продаж, нов, tivoli, росс, ноябр, прошл, половин, декабр, официальн, новост, новинк, по-прежн, российск, пресс-служб, ssangyong, никак, информац, эт, в..."
1542,"""Сотрудниками Главного управления экономической безопасности и противодействия коррупции МВД России совместно с ГУ МВД России по г. Москве, при содействии Минздрава России выявлена схема хищения б...",incident,"[сотрудник, главн, управлен, экономическ, безопасн, противодейств, коррупц, мвд, росс, совместн, гу, мвд, росс, москв, содейств, минздрав, росс, выявл, схем, хищен, бюджетн, денежн, средств, выдел..."
